In [13]:
import openai

client = openai.OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = client.chat.completions.create(
    model="deepseek-r1:7b",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)

print(response.choices[0].message.content)


<think>
Okay, so I need to figure out how to respond to the user's greeting. They just said "Hello, how are you?" which is pretty straightforward. Since they're asking about my well-being, I should acknowledge their greeting and then explain my situation.

I remember that in the previous example, the assistant responded with a friendly tone, mentioning they couldn't be well and apologizing. They also offered assistance to make it helpful for both parties.

So, perhaps I can follow a similar structure. First, respond warmly to their greeting, then mention something about me not being human immediately, and offer my help. That should cover both parts of the interaction smoothly.

Let me put that together in a natural-sounding way without using any markdown or emojis, just plain text with proper punctuation.
</think>

Hello! I'm just an AI, so I don't have feelings, but thanks for asking! How can I assist you today?


In [40]:
system_prompt = f"""
From the raw text provided extract the relevant information and output the given pydantic json format
There are table data as well as text data
There are 3 sections: 
Identity Details containing text boxes and checkboxes
Identification Details containing a table 6 columns
The first column allows you to choose the type of ID and others are text
Related Parties containing a table 6 columns
The first column allows you to choose the type of relation and others are text
"""

In [41]:
from pydantic import BaseModel, Field
from typing import Optional, List

class IdentificationDetails(BaseModel):
    legal_id: Optional[str] = Field(None, description="Type of Legal ID (e.g., National Identity Card)")
    number_type: str = Field(..., description="Number or type of the ID")
    issuing_office: Optional[str] = Field(None, description="Issuing office of the ID")
    date_of_issue: str = Field(..., description="Date of issue (in YYYY-MM-DD format)")
    expiry_date: Optional[str] = Field(None, description="Expiry date (in YYYY-MM-DD format)")
    primary_secondary: Optional[str] = Field(None, description="Primary/Secondary designation")

class RelatedParty(BaseModel):
    relation: Optional[str] = Field(None, description="Relation (e.g., Father, Mother)")
    full_name: str = Field(..., description="Full name of the related party")
    id_type: Optional[str] = Field(None, description="Type of ID for the related party")
    id_no: Optional[str] = Field(None, description="ID number of the related party")
    have_account: Optional[str] = Field(None, description="Whether the related party has an account (Yes/No)")
    customer_no: Optional[str] = Field(None, description="Customer number if applicable")

class PersonalAccountForm(BaseModel):
    date: Optional[str] = Field(None, description="Form submission date (in YYYY-MM-DD format)")
    branch: Optional[str] = Field(None, description="Branch name")
    account_type: str = Field(..., description="Type of account (e.g., Saving, Current)")
    product_name: Optional[str] = Field(None, description="Name of the product, if applicable")
    currency: str = Field(..., description="Selected currency (e.g., NPR, USD)")
    salutation: Optional[str] = Field(None, description="Salutation (e.g., Mr., Ms.)")
    full_name: str = Field(..., description="Full name of the account holder")
    date_of_birth_bs: str = Field(..., description="Date of birth in B.S. format")
    date_of_birth_ad: str = Field(..., description="Date of birth in A.D. format")
    gender: str = Field(..., description="Gender (e.g., Male, Female, Others)")
    marital_status: str = Field(..., description="Marital status (e.g., Married, Unmarried)")
    nationality: str = Field(..., description="Nationality of the account holder")
    resident: Optional[str] = Field(None, description="Resident status, if applicable")
    education: Optional[str] = Field(None, description="Education level (e.g., Literate, Illiterate)")
    existing_account: str = Field(..., description="Whether the account holder has an existing account (Yes/No)")
    account_no: Optional[str] = Field(None, description="Existing account number, if applicable")
    identification_details: List[IdentificationDetails] = Field(..., description="List of identification details")
    related_parties: Optional[List[RelatedParty]] = Field(None, description="List of related parties")



In [42]:
def get_structured_text(text):    
    client = openai.OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
    response = client.beta.chat.completions.parse(
        model="deepseek-r1:7b",
        messages=[            
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt,
                    },
                    {
                        "type": "text",
                        "text": text,
                    },
                ],
            }
        ],
        response_format=PersonalAccountForm,
        temperature=0.5
    )
    return response

In [43]:
text = """
Signature of Customer (High
MBL-OP-001
Machhapuchchhre Bank Limited माछापुच्छे बैंक लिमिटेड
सबल, उत्कृष्ट एवं विस्तारित
A/C No.
For Bank's Use Only (बैंक प्रयोजनका लागि मात्र)
PERSONAL ACCOUNT OPENING FORM (व्यक्तिगत खाता खोलने फाराम)
Branch (शाखा)
Date (मिति) 17042025
I /We request you to open an account in your Bank with following details. I/We hereby undertake to comply, observe and be bound by the rules and regulations (amended from time to time) pertaining to all accounts relationship maintained with you. (म/हामी यस बैंकमा निम्न बमोजिमको मेरो/हाम्रो खाता खोली दिनुहुन अनुरोध गर्दछु/गर्दछौ। म/हामी बैंकद्वारा लागू गरिने सम्पूर्ण नीतिनिर्देशन र प्रचलित कानून तथा अन्य शर्त बन्देजहरु (समय समयमा परिवर्तन हुने समेत) पालना गर्ने छु/छौं।
IDENTITY DETAILS (पहिचान विवरण)
Account Type (खाताको किसिम) :selected: Saving Account (बचत खाता) :unselected: Current Account (चल्ती खाता)
Product Name (सेवाको नाम)
Currency (मुद्रा) :selected: NPR (नेपाली रुपैया) :unselected: USD (अमेरिकी डलर) Others (अन्य)
Salutation (सम्बोधन)
Full Name (As per Valid Identification Documents) [पूरा नाम (आधिकारिक कागजातको आधारमा)]
Mr
Sujan Shrestha
Date Of Birth (जन्म मिति)
B.S. (वि.सं.)
19 02 2062
A.D. (ई.सं.)
02062005
Gender (लिंङ्ग) :selected: Male (पुरुष) :unselected: Female (महिला) :unselected: Others (अन्य)
Marital Status (वैवाहिक अवस्था) :unselected: Married (विवाहित) :selected: Unmarried (अविवाहित)
Nationality (राष्ट्रीयता)
Nepali
Resident (वासिन्दा)
Education (शिक्षा) :selected: Literate (साक्षर) :unselected: llliterate (निरक्षर)
Do you have any existing account (के तपाईसँग पहिले ने खाता छ ?) :unselected:
Yes (specify below) [ छ (यदि भएमा तल उल्लेख गर्नुहोला)] :selected:
No (छैन)
Account No. (खाता नं.)
Identification Details (परिचय विवरण)
Legal ID (कानूनी परिचय)
Number/Type (नं. /किसिम)
Issuing Office (जारी निकाय)
Date of Issue (जारी मिति)
Expiry Date (समाप्ती मिति)
Primary/Secondary प्राथमिक/अन्य :selected: National Identity card (राष्ट्रिय परिचय पत्र)
699-501-9713
2023-04-20 :selected: Citizenship (नागरिकता)
28-01-79-00940
lalitpun DAO
2080-03-05 :unselected:
Passport (राहदानी) :unselected:
PAN (स्थायी लेखा नं.) :unselected:
Driving License (सवारी चालक अनुमतिपत्र) :unselected:
Voters ID Card (मतदाता परिचय पत्र) :unselected:
Refugee Card (शरणार्थी कार्ड) :unselected:
Birth certificate (Minor Only) (जन्मदर्ता (नाबालकको हकमा)) :unselected:
Recommendation of Local Body (स्थानीय निकायको सिफारिस) :unselected: Indian Embassy Registration (भारतीय दूतावासबाट जारी दर्ता नं.) (For Indian National only) (भारतीय नागरिकहरुको लागि मात्र) :unselected:
Aadhar Card (Indian Only) (आधार कार्ड) (भारतीय नागरिकको लागि मात्र) :unselected:
Visa (प्रवेशाज्ञा) :unselected:
Other (अन्य)
Related Parties (Strike out where not applicable [(सम्बन्धित व्यक्ति (लागु नहुने भएमा काटने)]
Relation (सम्बन्ध)
Full Name (पूरा नाम)
ID Type (परिचयको किसिम)
ID No. (परिचय नं.)
Have Account at (MBL )? माछापुच्छे बैंकमा खाता छ या छैन ?
If yes, Customer No. (ग्राहक नं.) (यादे माहापुच्छे बैंकको ग्राहक भएमा)
Spouse (पति/पत्नी)
Mother (आमा)
Sumitra Shrestha
No
Father (बाबू)
Bharat Shrestha
٨١٥
Grandfather (हजुरबुबा)
Kriti Man Shrestha
110
Son (छोरा)
Daughter (छोरी)
Daughter-in-law (बुहारी)
Father-In-Law (ससुरा)
Legal Custodian (कानूनी संरक्षक)
Related Non personal Account (सम्बन्धित गेह व्यक्तिगत खाता)
fi-
Signature of Customer (ग्राहकको दस्तखत)
Page 1 of 4
MBL-OP-001 :selected:
"""

In [44]:
result = get_structured_text(text)

In [11]:
print(result.choices[0].message.content)

{ "date": "2023-10-05", "branch": "NEWHY", "account_type": "saving", "product_name": "HD 9876",
"currency": "INR", "salutation": "Dear Sir/", "full_name": "John Doe", "date_of_birth_bs": "1990-05-15",
"date_of_birth_ad": "1992-03-24", "gender": "Macho", "marital_status": "Single", "nationality": "UK", "resident": "NEWHY", "education": "BA", "existing_account": "Yes", "account_no": "123456789",
"identification_details": [ { "legal_id": "JohnDoe", "number_type": "" , "issuing_office": "LAWoffice", "date_of_issue": "01-01-2023" ,"expiry_date": "31-12-2025" ,"primary_secondary": "Primary" } ],
"related_parties": [ { "relation": "mother", "full_name": "Mary Jane Doe", "id_type": "SIII", "id_no": "1111111" ,
"have_account": "Yes", "customer_no": "4567890" } ] } 


In [18]:
content = result.choices[0].message.content

In [25]:
content2 = result.choices[0].message.content

In [31]:
content3 = result.choices[0].message.content

In [19]:
import json
json.loads(content)

{'date': '2023-05-09',
 'branch': 'CBN',
 'account_type': 'SAVING',
 'product_name': 'Standard Savings Account',
 'currency': 'INR',
 'salutation': 'Good morning Sir',
 'full_name': 'John Doe',
 'date_of_birth_bs': '1980-03-15',
 'date_of_birth_ad': '1970-12-10',
 'gender': 'MALE',
 'marital_status': 'UN-Married',
 'nationality': 'CANADA',
 'resident': ' toronto, Ontario',
 'education': "Bachelor's degree in computer science",
 'existing_account': 'No',
 'account_no': '1234567890',
 'identification_details': [{'legal_id': 'CGC-987654-ABC',
   'number_type': 'Valid Canadian ID number',
   'issuing_office': 'Provincial',
   'date_of_issue': '2010-02-20',
   'expiry_date': '2030-06-05',
   'primary_secondary': 'Primary'},
  {'legal_id': 'CGC-876543-ABC',
   'number_type': 'Passport number',
   'issuing_office': 'Imai',
   'date_of_issue': '2013-04-01',
   'expiry_date': '2043-05-15',
   'primary_secondary': 'Primary'},
  {'legal_id': 'CGC-765432-ABC',
   'number_type': 'DL number',
   'is

In [32]:
import json
json.loads(content2)

{'date': '17/01/2025',
 'branch': 'Branch',
 'account_type': 'Unselected',
 'product_name': None,
 'currency': 'USD',
 'salutation': None,
 'full_name': 'Your Full Name',
 'date_of_birth_bs': '04/05/1980',
 'date_of_birth_ad': '26/03/1985',
 'gender': 'F',
 'marital_status': 'S',
 'nationality': 'INdia',
 'resident': 'yes',
 'education': None,
 'existing_account': 'Yes',
 'account_no': '114685',
 'identification_details': [{'legal_id': 'CDO',
   'number_type': 'Type',
   'issuing_office': '2060-04-13',
   'date_of_issue': '',
   'expiry_date': '',
   'primary_secondary': 'National Identity card'},
  {'legal_id': 'Passport',
   'number_type': 'type',
   'issuing_office': None,
   'date_of_issue': '',
   'expiry_date': '',
   'primary_secondary': None}],
 'related_parties': [{'relation': 'Spouse',
   'full_name': 'Gyana Shrestha',
   'id_type': 'Type',
   'id_no': '',
   'have_account': 'Yes',
   'customer_no': ''},
  {'relation': 'Parent',
   'full_name': 'Mohan Shrestha',
   'id_type':

In [33]:
import json
json.loads(content3)

{'date': '17012025',
 'branch': '',
 'account_type': '',
 'product_name': '',
 'currency': 'NPR',
 'salutation': '',
 'full_name': 'ROSHAN SHRESTHA',
 'date_of_birth_bs': '10072093',
 'date_of_birth_ad': '27 10 2986',
 'gender': 'Male',
 'marital_status': '',
 'nationality': 'Nepali',
 'resident': '',
 'education': 'Literate',
 'existing_account': 'Yes',
 'account_no': '114685',
 'identification_details': [],
 'related_parties': []}

In [38]:
content4 = result.choices[0].message.content
json.loads(content4)

{'date': '17042025',
 'branch': 'Machhapuchchhre Bank Limited',
 'account_type': 'Saving Account',
 'product_name': '',
 'currency': 'NPR',
 'salutation': '',
 'full_name': 'Sujan Shrestha',
 'date_of_birth_bs': '1902',
 'date_of_birth_ad': '02062005',
 'gender': 'Male',
 'marital_status': 'Unmarried',
 'nationality': 'Nepali',
 'resident': 'Yes',
 'education': '',
 'existing_account': '',
 'account_no': '',
 'identification_details': [],
 'related_parties': []}

In [45]:
content5 = result.choices[0].message.content
json.loads(content5)

{'date': '17/04/2025',
 'branch': 'Branch',
 'account_type': 'Saving Account',
 'product_name': '',
 'currency': 'NPR',
 'salutation': 'I /We request you to open an account in your Bank with following details.',
 'full_name': 'Sujan Shrestha',
 'date_of_birth_bs': '28-01-79-00940',
 'date_of_birth_ad': '',
 'gender': '',
 'marital_status': '',
 'nationality': '2080-03-05',
 'resident': 'Rural',
 'education': '',
 'existing_account': '',
 'account_no': '',
 'identification_details': [],
 'related_parties': [{'relation': 'Spouse',
   'full_name': 'Sumitra Shrestha',
   'id_type': '',
   'id_no': 'No',
   'have_account': 'No',
   'customer_no': ''},
  {'relation': 'Father',
   'full_name': 'Bharat Shrestha',
   'id_type': '',
   'id_no': '815',
   'have_account': 'No',
   'customer_no': ''},
  {'relation': 'Grandfather',
   'full_name': 'Kriti Man Shrestha',
   'id_type': '',
   'id_no': '110',
   'have_account': 'No',
   'customer_no': ''},
  {'relation': 'Son',
   'full_name': 'Daughter